In [4]:
%%writefile matrixMul.cu

#include <stdio.h>
#include <cstdlib>

#define N 64 // Size of the square matrices


__global__ void matrixMultiplication(int *A, int *B, int *C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int sum = 0;
        for (int k = 0; k < N; ++k) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main() {
    int *A, *B, *C;
    int *d_A, *d_B, *d_C;
    size_t size = N * N * sizeof(int);
    cudaError_t cudaStatus;

    A = (int*)malloc(size);
    B = (int*)malloc(size);
    C = (int*)malloc(size);

    for (int i = 0; i < N ; ++i) {
      for(int j=0;j<N;j++){
        A[i*N+j] = rand()%10;
        B[i*N+j] = rand()%10;
    }
    }


    // Allocate memory for matrices A, B, and C on the device
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);


    // Copy matrices A and B from host to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(16, 16); // For simplicity, using fixed block size
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Launch kernel to perform matrix multiplication
    matrixMultiplication<<<gridDim, blockDim>>>(d_A, d_B, d_C);
    cudaDeviceSynchronize();

    // Copy result matrix C from device to host
    cudaStatus = cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);
    if (cudaStatus != cudaSuccess) {
    fprintf(stderr, "cudaMemcpy failed for result matrix C: %s\n", cudaGetErrorString(cudaStatus));
    exit(EXIT_FAILURE);
    }

    bool flag=1;

    for(int i=0;i<N && flag;i++){
      for(int j=0;j<N && flag;j++){
        int temp=0;
        for(int k=0;k<N;k++){
          temp+=(A[i*N+k]*B[k*N+j]);
        }
        if(temp!=C[i*N+j]){
          flag=0;
        }
      }
    }

    if(flag) printf("Multiplication is correctly performed by GPU.");
    else printf("Multiplication operation failed!!!");


    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(A);
    free(B);
    free(C);

    return 0;
}

Writing matrixMul.cu


In [5]:
!nvcc matrixMul.cu -o matrixMul

In [6]:
!./matrixMul

Multiplication is correctly performed by GPU.